# ODN Data Processing

This cell performs the following tasks:
1. Imports necessary libraries (pandas and os)
2. Defines the path to the ODN data folder
3. Reads all Excel files in the folder
4. Combines data from all files into a single DataFrame
5. Filters the data for Montevideo
6. Saves the filtered data to a CSV file

The resulting DataFrame `montevideo_odn_data` contains all ODN data for Montevideo.

In [ ]:
import pandas as pd
import os

# Definir la ruta a la carpeta ODN
odn_folder = r'[ODN_DATA_FOLDER]'

# Crear una lista para almacenar los datos de todos los archivos
all_data = []

# Recorrer todos los archivos en la carpeta ODN
for file_name in os.listdir(odn_folder):
    # Verificar si el archivo es un Excel moderno (xlsx)
    if file_name.endswith('.xlsx'):
        # Definir la ruta completa del archivo
        file_path = os.path.join(odn_folder, file_name)
        # Leer el archivo Excel, con el encabezado en la fila 9
        df = pd.read_excel(file_path, header=8, usecols='A:I')
        # Agregar los datos a la lista
        all_data.append(df)

# Combinar todos los datos en un solo DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Filtrar los datos donde la columna DEPTO es Montevideo
montevideo_odn_data = combined_data[combined_data['DEPTO'] == 'Montevideo']

# Mostrar los primeros registros del DataFrame filtrado
montevideo_odn_data.head()

montevideo_odn_data.to_csv('montevideo_odn_dataset.csv', index=False)


# Data Cleaning and Transformation

This cell performs the following tasks:
1. Renames the 'CONVOCATORIA' column to 'PARTIDO'
2. Defines a function to clean and format party names
3. Applies the cleaning function to the 'PARTIDO' column
4. Removes the 'ACTO' column if it exists
5. Displays the first few rows of the modified DataFrame

The resulting DataFrame `montevideo_odn_data` now has cleaned and standardized party names.

In [ ]:
# Renombrar la columna 'CONVOCATORIA' a 'PARTIDO'
montevideo_odn_data.rename(columns={'CONVOCATORIA': 'PARTIDO'}, inplace=True)

# Función para modificar los valores de la columna 'PARTIDO'
def modificar_partido(valor):
    # Separar el valor por '_-_' y tomar la primera parte
    partido = valor.split('_-_')[0]
    # Reemplazar '_' por espacios y agregar 'Partido' al inicio
    partido = partido.replace('_', ' ')
    # Eliminar el sufijo ' O.D.N.' si existe
    if 'O.D.N.' in partido:
        partido = partido.replace(' O.D.N.', '')
    return partido

# Aplicar la función a la columna 'PARTIDO'
montevideo_odn_data['PARTIDO'] = montevideo_odn_data['PARTIDO'].apply(modificar_partido)

# Verificar si la columna 'ACTO' existe y eliminarla si es el caso
if 'ACTO' in montevideo_odn_data.columns:
    montevideo_odn_data.drop(columns=['ACTO'], inplace=True)

# Mostrar los primeros registros del DataFrame modificado
montevideo_odn_data.head()


# Save Modified ODN Dataset

This cell performs the following task:
1. Saves the modified `montevideo_odn_data` DataFrame to a CSV file named 'montevideo_odn_dataset_modificado.csv'

The resulting CSV file contains the cleaned and transformed ODN data for Montevideo, without including the index column.

In [ ]:
montevideo_odn_data.to_csv('montevideo_odn_dataset_modificado.csv', index=False)

# Add Zone Information to ODN Data

This cell performs the following tasks:
1. Reads a reference CSV file for mapping series to zones
2. Creates a dictionary to map series to zones
3. Defines a function to obtain the zone for each series
4. Applies the function to add a 'ZONA' column to the ODN dataset
5. Displays the first few rows of the modified DataFrame
6. Saves the final DataFrame with zone information to a CSV file

The resulting CSV file 'montevideo_odn_dataset_con_zona.csv' contains the ODN data for Montevideo with added zone information.

In [ ]:
import pandas as pd

# Leer el archivo CSV de referencia
ref_df = pd.read_csv('votos_por_barrio_pn_mapeado_odn.csv', delimiter=';')

# Asegurarse de que la columna 'SERIE' sea de tipo string
ref_df['SERIE'] = ref_df['SERIE'].astype(str)

# Crear un diccionario para mapear series a zonas
series_to_zona = {}
for index, row in ref_df.iterrows():
    series = row['SERIE'].split()  # Dividir las series si están separadas por espacios
    for serie in series:
        series_to_zona[serie] = row['ZONA']

# Función para mapear serie a zona
def obtener_zona(serie):
    series = str(serie).split()  # Asegurarse de que la serie sea una cadena y dividirla
    zonas = [series_to_zona.get(s, 'Desconocido') for s in series]  # Obtener la zona para cada serie
    return ' '.join(set(zonas))  # Devolver las zonas únicas

# Aplicar la función al DataFrame existente (montevideo_odd_data)
montevideo_odn_data['ZONA'] = montevideo_odn_data['SERIES'].apply(obtener_zona)

# Mostrar los primeros registros del DataFrame modificado
print(montevideo_odn_data.head())

# Guardar el DataFrame modificado en un archivo CSV
montevideo_odn_data.to_csv('montevideo_odn_dataset_con_zona.csv', index=False)
